In [1]:
#Homework 5  1st question about loading and preprocessing the data.

In [2]:
!pip install pyspark
from subprocess import check_output
from pyspark.sql import SparkSession
import pandas as pd 
import numpy as np
from pyspark.sql.functions import (count, col)
from pyspark.ml import Pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285413 sha256=982579bdd27acd29c8e4b8b27ed8cabf904abc3ead641762afbfb6244b2ebf20
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!kaggle datasets download -d uciml/adult-census-income


Traceback (most recent call last):
  File "/opt/conda/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/opt/conda/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/opt/conda/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()
df = spark.read.format('csv').option('header', True).load('/kaggle/input/adult-census-income',header=True, inferSchema=True)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/18 14:51:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education.num: integer (nullable = true)
 |-- marital.status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital.gain: integer (nullable = true)
 |-- capital.loss: integer (nullable = true)
 |-- hours.per.week: integer (nullable = true)
 |-- native.country: string (nullable = true)
 |-- income: string (nullable = true)



In [6]:
df.toPandas().head(10)


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
5,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K
6,38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K
7,74,State-gov,88638,Doctorate,16,Never-married,Prof-specialty,Other-relative,White,Female,0,3683,20,United-States,>50K
8,68,Federal-gov,422013,HS-grad,9,Divorced,Prof-specialty,Not-in-family,White,Female,0,3683,40,United-States,<=50K
9,41,Private,70037,Some-college,10,Never-married,Craft-repair,Unmarried,White,Male,0,3004,60,?,>50K


In [7]:
education_num = col("`education.num`")
capital_gain = col("`capital.gain`")
capital_loss = col("`capital.loss`")
hours_per_week = col("`hours.per.week`")
marital_status = col("`marital.status`")
native_country = col("`native.country`")

# rename dot-name columns
df = df.withColumn("education_num", education_num).drop(education_num)\
    .withColumn("capital_gain", capital_gain).drop(capital_gain)\
    .withColumn("capital_loss", capital_loss).drop(capital_loss)\
    .withColumn("hours_per_week", hours_per_week).drop(hours_per_week)\
    .withColumn("marital_status", marital_status).drop(marital_status)\
    .withColumn("native_country", native_country).drop(native_country)

In [8]:
num_vars = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]

In [9]:
cat_vars = ["workclass", "education", "marital_status", "occupation", 
            "relationship", "race", "sex", "native_country"]

In [10]:
def countNull(df, var):
    return df.where(df[var].isNull()).count()

all_cols = num_vars + cat_vars
{var: countNull(df, var) for var in all_cols}

{'age': 0,
 'fnlwgt': 0,
 'education_num': 0,
 'capital_gain': 0,
 'capital_loss': 0,
 'hours_per_week': 0,
 'workclass': 0,
 'education': 0,
 'marital_status': 0,
 'occupation': 0,
 'relationship': 0,
 'race': 0,
 'sex': 0,
 'native_country': 0}

In [11]:

from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler

stages = []
for cat_var in cat_vars:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(
        inputCol=cat_var, 
        outputCol=cat_var+"_indx")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    encoder = OneHotEncoder(
        inputCols=[stringIndexer.getOutputCol()], 
        outputCols=[cat_var + "_vec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

In [12]:
label_indx = StringIndexer(
    inputCol="income", 
    outputCol="label")
stages += [label_indx]


In [13]:
assembler_inputs = [c+"_vec" for c in cat_vars] + num_vars
assembler = VectorAssembler(
    inputCols=assembler_inputs, 
    outputCol="features")
stages += [assembler]

In [14]:
pipeline = Pipeline(stages=stages)

In [15]:
df = pipeline.fit(df).transform(df)

In [16]:
selectedcols = ["label", "features"] + all_cols
df = df.select(selectedcols)
df.toPandas().head(10)

23/07/18 14:52:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,label,features,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass,education,marital_status,occupation,relationship,race,sex,native_country
0,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",90,77053,9,0,4356,40,?,HS-grad,Widowed,?,Not-in-family,White,Female,United-States
1,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",82,132870,9,0,4356,18,Private,HS-grad,Widowed,Exec-managerial,Not-in-family,White,Female,United-States
2,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",66,186061,10,0,4356,40,?,Some-college,Widowed,?,Unmarried,Black,Female,United-States
3,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",54,140359,4,0,3900,40,Private,7th-8th,Divorced,Machine-op-inspct,Unmarried,White,Female,United-States
4,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",41,264663,10,0,3900,40,Private,Some-college,Separated,Prof-specialty,Own-child,White,Female,United-States
5,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",34,216864,9,0,3770,45,Private,HS-grad,Divorced,Other-service,Unmarried,White,Female,United-States
6,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",38,150601,6,0,3770,40,Private,10th,Separated,Adm-clerical,Unmarried,White,Male,United-States
7,1.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",74,88638,16,0,3683,20,State-gov,Doctorate,Never-married,Prof-specialty,Other-relative,White,Female,United-States
8,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...",68,422013,9,0,3683,40,Federal-gov,HS-grad,Divorced,Prof-specialty,Not-in-family,White,Female,United-States
9,1.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",41,70037,10,0,3004,60,Private,Some-college,Never-married,Craft-repair,Unmarried,White,Male,?


In [17]:
(train, test) = df.randomSplit([0.7, 0.3], seed=100)
print(train.count())
print(test.count())

22832


9729


In [18]:
from pyspark.ml.classification import (LogisticRegression, DecisionTreeClassifier,RandomForestClassifier, NaiveBayes, GBTClassifier)

#Logistic regression model
logr = LogisticRegression(
    maxIter = 10,
    regParam = 0.05,
    labelCol="label",
    featuresCol="features"
)

#KNN
# k_nn = KNNClassifer(
#     numTrees = 10,
#     labelCol = "label",
#     featuresCol="features"
# )

 #decision tree model
d_tree = DecisionTreeClassifier(
    maxDepth = 10,
    labelCol = "label",
    featuresCol="features"
)
#NaiveBayes
n_bayes = NaiveBayes(
#     maxDepth = 10,
    labelCol = "label",
    featuresCol="features"
)

g_bt = GBTClassifier(
    maxDepth = 10,
    labelCol = "label",
    featuresCol="features"
)

# random forest model
r_forest = RandomForestClassifier(
    numTrees = 10,
    labelCol = "label",
    featuresCol="features"
)


# fit models
lr_model = logr.fit(train)
dt_model = d_tree.fit(train)
rf_model = r_forest.fit(train)
# knn_model = k_nn.fit(train)
nb_model = n_bayes.fit(train)
gbt_model = g_bt.fit(train)


23/07/18 14:52:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/07/18 14:53:12 WARN DAGScheduler: Broadcasting large task binary with size 1014.1 KiB
23/07/18 14:53:12 WARN DAGScheduler: Broadcasting large task binary with size 1013.9 KiB
23/07/18 14:53:12 WARN DAGScheduler: Broadcasting large task binary with size 1014.4 KiB
23/07/18 14:53:12 WARN DAGScheduler: Broadcasting large task binary with size 1015.0 KiB
23/07/18 14:53:12 WARN DAGScheduler: Broadcasting large task binary with size 1016.1 KiB
23/07/18 14:53:12 WARN DAGScheduler: Broadcasting large task binary with size 1017.8 KiB
23/07/18 14:53:13 WARN DAGScheduler: Broadcasting large task binary with size 1021.1 KiB
23/07/18 14:53:13 WARN DAGScheduler: Broadcasting large task binary with size 1027.5 KiB
23/07/18 14:53:13 WARN DAGScheduler: Broadcasting large task binary with size 1039.0 KiB
23/07/18 14:53:13 WARN DAGScheduler: Broadcasting large task binary with size 1057.3 KiB
23/

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# model evaluator
def testModel(model, df):
    pred = model.transform(df)
    evaluator = BinaryClassificationEvaluator(labelCol="label")
    return evaluator.evaluate(pred)

# accuracy output
models = {
    "Logistic regression": lr_model,
    "Decision tree": dt_model,
    "Random forest": rf_model,
    "NaiveBayes": nb_model,
    "GradientBoostTrees":gbt_model
}

# model performance comparisson
{model_name: testModel(model, test) for model_name,model in models.items()}

23/07/18 14:53:33 WARN DAGScheduler: Broadcasting large task binary with size 1546.7 KiB


{'Logistic regression': 0.8943934992297966,
 'Decision tree': 0.8016051475917901,
 'Random forest': 0.8748910737321414,
 'NaiveBayes': 0.36452231538246793,
 'GradientBoostTrees': 0.9020230476840314}

In [20]:
predictions = lr_model.transform(test)
predictions.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: integer (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education_num: integer (nullable = true)
 |-- capital_gain: integer (nullable = true)
 |-- capital_loss: integer (nullable = true)
 |-- hours_per_week: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- education: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- native_country: string (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [21]:
inspect_cols = ["label", "probability", "prediction", "age", "occupation", "sex", "hours_per_week"]
predictions.select(inspect_cols).show(15)

+-----+--------------------+----------+---+--------------+------+--------------+
|label|         probability|prediction|age|    occupation|   sex|hours_per_week|
+-----+--------------------+----------+---+--------------+------+--------------+
|  0.0|[0.29587245995467...|       1.0| 36|Prof-specialty|  Male|            60|
|  0.0|[0.66874817876628...|       0.0| 32|Prof-specialty|  Male|            40|
|  0.0|[0.56812374211772...|       0.0| 33|Prof-specialty|  Male|            60|
|  0.0|[0.64949942482009...|       0.0| 39|Prof-specialty|  Male|            40|
|  0.0|[0.61863901973377...|       0.0| 39|Prof-specialty|  Male|            45|
|  0.0|[0.60635940124204...|       0.0| 50|Prof-specialty|  Male|            40|
|  0.0|[0.60157267853152...|       0.0| 51|Prof-specialty|  Male|            40|
|  0.0|[0.60306827555701...|       0.0| 60|Prof-specialty|  Male|            30|
|  0.0|[0.70128008415338...|       0.0| 34|Prof-specialty|  Male|            40|
|  0.0|[0.92967610098900...|